# The Battle of Neighbourhood . Week 1 Data Section 

Based on definition of our problem, factors that will influence our decision are

* Number of existing Gyms in the neighborhood
* Number of and distance to Yoga Centers in the neighborhood.
* Number of Homes,Offices,Colleges,Commercial activity in the neighborhood. 

Bangalore has Several suburbs and stellite locations which may not be ideal 0 to start new business.
So first we will determine the effective subset of Bangalore for our analysis. 

Then we will define closely packed grids of radius around 500 metres for our analysis

We will use the following data sources to generate the required information:

* The suburbs in Bangalore : We will use information in wikipedia to get the list of suburbs in Bangalore. 
* Then we will determine an effective radius which covers maximum number of suburbs. 
* Then we generate the grid centres/locations and determine the their approximate addresses using the Opencage API
* In general Spatial coordinates and reverse geocoding will be performed using **Opencage API**.
* Number of Gyms,Yoga centers and amenities viz homes,offices,colleges,commercial centers in every location  will be obtained using **Foursquare API**. 
* Also we will need sveral Python Libraries like, Pandas, Numpy, Folium to be used for our Analysis.





# Lets first Install and import the required Libraries 

In [2]:
#!pip install opencage
#!pip install shapely
#!pip install pyproj
#!pip install folium

import pandas as pd
import numpy as np 
import folium

import pyproj
import math
import pickle


from sklearn.cluster import KMeans
import requests
from pandas.io.json import json_normalize

# Lets get the list of suburbs in Bangalore 

In [4]:
web_data = pd.read_html('https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Bangalore',header=0)
#number_of_tables = len(web_data1.str.contains('Name'))
#print(number_of_tables)
frames = []
 
for i in range(0,8):
    frames.append(web_data[i])

bangalore = pd.concat(frames)
bangalore.drop(['Image','Summary'],axis=1,inplace = True)
bangalore.reset_index(drop=True,inplace=True)
bangalore["Latitude"] = np.zeros(len(bangalore),dtype = float)
bangalore["Longitude"] = np.zeros(len(bangalore),dtype = float)
bangalore.head()

,Name,Latitude,Longitude
0,Cantonment area,0.0,0.0
1,Domlur,0.0,0.0
2,Indiranagar,0.0,0.0
3,Jeevanbheemanagar,0.0,0.0
4,Malleswaram,0.0,0.0


## Now using the OpenCage API we determine spatial coordinates of the suburbs

In [5]:
# determine the Spatial coordinates of Bangalore 
from opencage.geocoder import OpenCageGeocode
from pprint import pprint
import numpy as np
blr_lat = 0.0
blr_long = 0.0

geocoder = OpenCageGeocode(GEO_KEY)
query = u'Bangalore,Karnataka,India'
results = geocoder.geocode(query)
blr_lat = results[0]['geometry']['lat']
blr_long = results[0]['geometry']['lng']
#print(results[0]['geometry']['lat'],results[0]['geometry']['lng'])
print("Coordinates of Bangalore ",blr_lat,blr_long)
blr_center = [blr_lat,blr_long]

Coordinates of Bangalore  12.9791198 77.5912997


In [6]:
for i in bangalore.index:
    query = u''+str(bangalore['Name'][i])+' Bangalore,Karnataka,India'
    results = geocoder.geocode(query)
    bangalore.at[i,'Latitude'] = results[0]['geometry']['lat']
    bangalore.at[i,'Longitude']= results[0]['geometry']['lng']

bangalore.head()

,Name,Latitude,Longitude
0,Cantonment area,13.019567,77.509589
1,Domlur,12.962467,77.638196
2,Indiranagar,12.973291,77.640467
3,Jeevanbheemanagar,12.971940,77.593690
4,Malleswaram,13.016341,77.558664


In [17]:
blr_lat = np.mean(bangalore['Latitude'])
blr_long = np.mean(bangalore['Longitude'])
blr_center = [blr_lat,blr_long]
print(blr_center)

[12.972315783076922, 77.5984514523077]


Now we proceed to create equally spaced circular grid with centeres 1 KM apart. In effect each circular grid of radius 500m.

We will need to transform the spatial coordinates into Spherical cartesan coordinates, which will allow to calculate distances in meters 
To accurately calculate distances we need to create our grid of locations in Cartesian 2D coordinate system which allows us to calculate distances in meters and not in degrees  
So we will create transform functions convert between WGS84 spherical coordinate system (latitude/longitude degrees) and UTM Cartesian coordinate system (X/Y coordinates in meters).

In [7]:
# Transforamtion Routines : To be used to convert Spatial Coordinates to Catersian Coordinates and Vice-versa
import pyproj

# Converts given longitude and latitude to Cartesian Coordinates 
def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=43, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]

# Converts given Cartesian Coordinates into latitude and Longitude 
def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=43, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]

# Returns the cartesian distance between two points. 
def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

print('Coordinate transformation check')
print('-------------------------------')
print('Bangalore center longitude={}, latitude={}'.format(blr_center[1], blr_center[0]))
x, y = lonlat_to_xy(blr_center[1], blr_center[0])
print('Bangalore center UTM X={}, Y={}'.format(x, y))
lo, la = xy_to_lonlat(x, y)
print('Bangalore center longitude={}, latitude={}'.format(lo, la))


Coordinate transformation check
-------------------------------
Bangalore center longitude=77.5912997, latitude=12.9791198
Bangalore center UTM X=781114.3002504944, Y=1436255.120232668
Bangalore center longitude=77.5912997, latitude=12.979119799999996


The below steps are used to identify effective radius of Bangalore to be used for analysis. 
* We first determine the distance of the farthest suburb from the Bangalore centre
* We determine the average distance of the suburbs from Bangalore Suburb.
* We determine the media distance of the suburbs from the center.
* Use each of the above as effective radius we plot the suburbs on the Folium maps to find the smallest effective radius 
   that covers maximum number of suburbs 


In [8]:
# Compute distance of the farthest Neighbourhood from the center
blr_center_x, blr_center_y = lonlat_to_xy(blr_center[1], blr_center[0])
Dmax = 0.0 
dist = np.zeros(len(bangalore),dtype =float)

for i in bangalore.index:
    n_x,n_y = lonlat_to_xy(bangalore['Longitude'][i],bangalore['Latitude'][i])
    d   = calc_xy_distance(blr_center_x, blr_center_y, n_x, n_y)
    dist[i] = d
    if d > Dmax:
        Dmax=d
max_dist = int(Dmax)
avg_dist = int(np.mean(dist))
median_dist = int(np.median(dist))

print(avg_dist)
print(median_dist)
print(max_dist)

bangalore['Distance from Center'] = dist

8349
7632
38813


In [9]:
# We map the suburbs and see which radius best brings in maximum number of suburbs 
map_bangalore = folium.Map(location=blr_center, zoom_start=10)
folium.Circle(blr_center,radius = avg_dist,color ='blue', popup='Bangalore').add_to(map_bangalore)
folium.Circle(blr_center,radius = median_dist,color ='red', popup='Bangalore').add_to(map_bangalore)
folium.Circle(blr_center,radius = max_dist,color ='blue', popup='Bangalore').add_to(map_bangalore)
for lat, lon in zip(bangalore['Latitude'], bangalore['Longitude']):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_bangalore) 
map_bangalore

We observe that a minimum radius of 7.3 Km will still keep lot of suburbs out of recknoning . 
So as a traial we take a mean of the three distances and see if the computed result is numerically effective 

In [10]:
# Seeing above we see while taking max distance brings of lot of areas under Bangalore Rural into our analysis . 
# Lets compute an average of the mean median and max to come to approximate radius that can be used to limit the area of our analysis
blr_radius = round((median_dist+max_dist+avg_dist)/3,-3)

print(blr_radius)
map_bangalore = folium.Map(location=blr_center, zoom_start=10)
folium.Circle(blr_center,radius = avg_dist,color ='blue', popup='Bangalore').add_to(map_bangalore)
folium.Circle(blr_center,radius = median_dist,color ='red', popup='Bangalore').add_to(map_bangalore)
folium.Circle(blr_center,radius = max_dist,color ='blue', popup='Bangalore').add_to(map_bangalore)
folium.Circle(blr_center,radius = 18000,color ='red', popup='Bangalore').add_to(map_bangalore)
for lat, lon in zip(bangalore['Latitude'], bangalore['Longitude']):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_bangalore) 
    
map_bangalore

18000.0


## Thus we see a radius of 18000 m or 18Km brings in  maximum number of suburbs thus we can exclude the rural areas
So will now limit our analysis within an area of roughly 1017 Square Kms

Now we proceed to create circular cell/grid spaced 1 KM apart . 
* The below algorithm basically divides a larger circle of given radius into smaller circles of given radius.
* These circular grids or cells will be basic unit of our analysis.
* For Visualization we shall plot each cell on the Folium maps 
* Using reverse geocoding we will get effective addresses of the location centers 
* We will use FourSquare API to get profile of each location. 

In [11]:
# The below algorithm divides the circular area of bangalore into uniform circular grids of 500m radius. 
# Each circle will be the smalleslt unit that will be analysed for Gym,Yoga Centers 
blr_center_x, blr_center_y = lonlat_to_xy(blr_center[1], blr_center[0]) # City center in Cartesian coordinates


x_min = blr_center_x - blr_radius
locality_radius = 500
x_step = 2*locality_radius
y_min = blr_center_y - blr_radius 
y_step = 2 * locality_radius
x_max  = blr_center_x + blr_radius
y_max  = blr_center_x + blr_radius
n_max  = int((2*blr_radius/x_step))

print(y_min,x_min)

latitudes = []
longitudes = []
distances_from_center = []
xs = []
ys = []
for i in range(0,n_max+1):
    y = y_min +  i*y_step
    for j in range(0,n_max+1):
        x = x_min +  j*x_step 
        distance_from_center = calc_xy_distance(blr_center_x, blr_center_y, x, y)
        if (distance_from_center <= 18001):
            lon, lat = xy_to_lonlat(x, y)
            latitudes.append(lat)
            longitudes.append(lon)
            distances_from_center.append(distance_from_center)
            xs.append(x)
            ys.append(y)

print(len(latitudes), 'candidate neighborhood centers generated.')

1418255.120232668 763114.3002504944
1009 candidate neighborhood centers generated.


In [12]:
map_bangalore = folium.Map(location=blr_center, zoom_start=10)
folium.Marker(blr_center, popup='Bangalore').add_to(map_bangalore)
folium.Circle(blr_center,radius = blr_radius,color ='red', popup='Bangalore').add_to(map_bangalore)
#for lat, lon in zip(bangalore['Latitude'], bangalore['Longitude']):
for lat, lon in zip(latitudes, longitudes):
    #folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_bangalore) 
    folium.Circle([lat, lon],radius=locality_radius,color='blue', fill=False).add_to(map_bangalore)
    #folium.Marker([lat, lon]).add_to(map_bangalore)
map_bangalore

The cells cover effectively the cover the area of Bangalore under study

Now we use the OpenCage API for reverse Geocoding to get effectives addresses of each Cell

In [18]:
from opencage.geocoder import InvalidInputError, RateLimitExceededError, UnknownError

def get_locality_address(api_key,lat,long):
    key = api_key
    try:
        results = geocoder.reverse_geocode(lat,long)
        if results and len(results):
           return (results[0]['formatted'])
    except RateLimitExceededError as ex:
           return None 

addr = get_locality_address(GEO_KEY,blr_center[0],blr_center[1]) 
print(addr)

Shantala Nagar, Bengaluru - 560001PH, Karnataka, India


In [20]:
bangalore_localities = []
for lat, lon in zip(latitudes, longitudes):
    local_addr = get_locality_address(GEO_KEY, lat, lon)
    if local_addr is None:
        local_addr = 'NO ADDRESS'
    local_addr = local_addr.replace(', India', '') # We don't need country part of address
    bangalore_localities.append(local_addr)

In [21]:
bangalore_localities[100:110]

['Ganakkal, Sompura - 560074, Karnataka',
 'Hemmigepura, Bengaluru - 560062, Karnataka',
 'Hemmigepura, Bengaluru - 560062, Karnataka',
 'Hemmigepura, Bengaluru - 560062, Karnataka',
 'Hemmigepura, Bengaluru - 560062, Karnataka',
 'Anjanapura, Bengaluru - 560062, Karnataka',
 'Anjanapura, Bengaluru - 560062, Karnataka',
 'Anjanapura, Bengaluru - 560083, Karnataka',
 'Gottigere Ward, Bengaluru - 560083, Karnataka',
 'Loyola P U College, Amruth Nagar Main Road, Gottigere Ward, Bengaluru - 560083, Karnataka']

In [26]:
# Now we create a composite dataframe having all the localities of Bangalore with addresses, coordinates, distance from center 
blr_locations = pd.DataFrame()
blr_locations['Address']  = bangalore_localities
blr_locations['Latitude'] = latitudes 
blr_locations['Longitudes'] = longitudes
blr_locations['X']  = xs
blr_locations['Y']  = ys
blr_locations['Distance from center'] = distances_from_center
#blr_locations = pd.DataFrame({'Address': bangalore_localities,
#                             'Latitude': latitudes,
#                             'Longitude': longitudes,
#                             'X': xs,
#                             'Y': ys,
#                             'Distance from center': distances_from_center})

blr_locations.head()

,Address,Latitude,Longitudes,X,Y,Distance from center
0,"Bannerughatta, Anekal",12.816517,77.589626,781114.30025,1.418255e+06,18000.000000
1,"Vajrahalli, Vaddarapalya - 560062, Karnataka",12.826000,77.543689,776114.30025,1.419255e+06,17720.045147
2,"Byatarayanadoddi, Anekal",12.825911,77.552895,777114.30025,1.419255e+06,17464.249197
3,"Byatarayanadoddi, Anekal",12.825821,77.562101,778114.30025,1.419255e+06,17262.676502
4,"Bannerughatta, Anekal",12.825731,77.571307,779114.30025,1.419255e+06,17117.242769


**Foursquare**

We use the Foursquare API to get information on GYMS/Fitness centers in each location. We will also determine the Yoga Centers in each location. 
Besides in general we shall fetch the number of Offices, Residential Locations, Colleges,Commercial Activity in each cell as well in entire Bangalore.Collectively these will be referred to as amenities. 
This will help us in accessing a cells profile.
* From the number of amenities we could infer the presence of community i.e presence of a customer base.
* This will aslo help us eliminate areas which are remote and sparesly populated. 


In [27]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']   

In [35]:
def get_local_feature(lat,lon,category,rradius):
    url =('https://api.foursquare.com/v2/venues/explore?client_id={}'
          '&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}').format(CLIENT_ID, 
                                                                  CLIENT_SECRET, 
                                                                  VERSION, 
                                                                  lat, 
                                                                  lon,
                                                                  category,  
                                                                  rradius,
                                                                  LIMIT)
        
    try:
        results = requests.get(url).json()['response']['groups'][0]['items']
        nearby_venues = json_normalize(results)
#       Filter the columns
        filtered_columns = ['venue.id','venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng','venue.location.distance']
        nearby_venues = nearby_venues.loc[:, filtered_columns]
# Filter the category for each row
        nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis = 1)
# Clean all column names
        nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
#     print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))
    except:
        nearby_venues = []
            
    return nearby_venues


In [29]:
# Getting the Distribution of Offices and Residences in Bangalore 

def get_location_profile(lat,lon,radius,limit,categories):
    
    profile = {}
    
    
    for category in categories:

        url =('https://api.foursquare.com/v2/venues/explore?client_id={}'
               '&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}').format(CLIENT_ID, 
                                                                  CLIENT_SECRET, 
                                                                  VERSION, 
                                                                  lat, 
                                                                  lon,
                                                                  category,  
                                                                  radius,
                                                                  limit)
        
        results = requests.get(url).json()['response']['groups'][0]['items']
        results = json_normalize(results)
        #       Filter the columns
        filtered_columns = ['venue.id','venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng','venue.location.distance']
        results = results.loc[:, filtered_columns]
# Filter the category for each row
        results['venue.categories'] = results.apply(get_category_type, axis = 1)
# Clean all column names
        results.columns = [col.split(".")[-1] for col in results.columns]
        
        profile[category] = results
    return profile
#        print('{} offices were returned by Foursquare.'.format(results.shape[0]))
            
    

In [33]:
categories =  [OFFICE,HOME,COLLEGE,COMMERCE]
blr_profile = get_location_profile(blr_center[0],blr_center[1],18000,1000,categories)
pr_xs = []
pr_ys = []
for category in categories:
    for i in blr_profile[category].index:
        x,y = lonlat_to_xy(blr_profile[category]['lng'][i],blr_profile[category]['lat'][i])
        pr_xs.append(x)
        pr_ys.append(y)
        
    blr_profile[category]['xs'] = pr_xs
    blr_profile[category]['ys'] = pr_ys
    pr_xs = []
    pr_ys = []

In [34]:
print ('Number of Offices :',len(blr_profile[OFFICE]))
print('Number of Residential locations :',len(blr_profile[HOME]))
print('Number of Centers with Commercial activities:',len(blr_profile[COMMERCE]))
print('Number of Colleges',len(blr_profile[COLLEGE]))

Number of Offices : 100
Number of Residential locations : 52
Number of Centers with Commercial activities: 100
Number of Colleges 41


In [36]:
# Lets go over all the localities and compose a table 
def compute_feature(lats,lons):
    gyms = {}
    yoga_centers = {}
    residences   = {}
    offices      = {}
    location_gyms = []
    location_yoga = []
    location_profile = []
    ofc_cnt       = 0
    cats    = [OFFICE,HOME,COMMERCE,COLLEGE]
    
    print('Analysizing the area around the localities')
    for lat,lon in zip(lats,lons):
    
        
        v_ofc    = get_local_feature(lat,lon,OFFICE,500)
        v_res    = get_local_feature(lat,lon,HOME,500)
        v_cmrc   = get_local_feature(lat,lon,COMMERCE,500)
        v_edu    = get_local_feature(lat,lon,COLLEGE,500)
        ofc_cnt  = len(v_ofc)
        res_cnt  = len(v_res)
        shp_cnt  = len(v_cmrc)
        clg_cnt  = len(v_edu)
        profile  = [ofc_cnt,res_cnt,shp_cnt,clg_cnt]
        location_profile.append(profile)
        
        v_gyms   = get_local_feature(lat,lon,GYM,550)
        local_gyms = []
        for i in range(0,len(v_gyms)):
            gym_name = v_gyms['name'][i]
            gym_vid  = v_gyms['id'][i]
            gym_cat  = v_gyms['categories'][i]
            gym_lat  = v_gyms['lat'][i]
            gym_lon  = v_gyms['lng'][i]
            gym_dist = v_gyms['distance'][i]
            x,y      = lonlat_to_xy(gym_lon, gym_lat)
            
            gym_d    = (gym_vid,gym_name,gym_cat,gym_lat,gym_lon,gym_dist,x,y)
            if gym_dist <= 500:
                local_gyms.append(gym_d)
                
            gyms[gym_vid] = gym_d
            
        location_gyms.append(local_gyms)    
            
        v_yoga   = get_local_feature(lat,lon,YOGA,550)
        local_ygc  = []
        for j in range(0,len(v_yoga)):
            yg_vid  = v_yoga['id'][j]
            yg_name = v_yoga['name'][j]
            yg_cat  = v_yoga['categories'][j]
            yg_lat  = v_yoga['lat'][j]
            yg_lon  = v_yoga['lng'][j]
            yg_dist = v_yoga['distance'][j]
            x,y      = lonlat_to_xy(yg_lon, yg_lat)
            yog_d     = (yg_vid,yg_name,yg_cat,yg_lat,yg_lon,yg_dist,x,y)
        
            if yg_dist <= 500:
                local_ygc.append(yog_d)
            yoga_centers[yg_vid] = yog_d
                
        location_yoga.append(local_ygc)
    
    return gyms,location_gyms,yoga_centers,location_yoga,location_profile        
        #v_office = get_locality_features(lat,lon,OFFICE)
        #for off in v_office:
        #v_homes  = get_locality_features(lat,lon,RESIDENCIES)

In [37]:
gyms,location_gyms,yoga_centers,location_yoga,location_profile = compute_feature(latitudes,longitudes)
print('Total number of Gyms:', len(gyms))
print('Total number of Yoga Centers:', len(yoga_centers))


Analysizing the area around the localities
Total number of Gyms: 674
Total number of Yoga Centers: 64


In [38]:
local_gym_count = [len(res) for res in location_gyms]
local_ygc_count = [len(yg) for yg in location_yoga ]
#print(len(local_gym_count))
blr_locations['Gyms in the Area'] = local_gym_count
blr_locations['Yoga Centers in Area'] = local_ygc_count 

In [39]:
blr_locations.head()

,Address,Latitude,Longitudes,X,Y,Distance from center,Gyms in the Area,Yoga Centers in Area
0,"Bannerughatta, Anekal",12.816517,77.589626,781114.30025,1.418255e+06,18000.000000,0,0
1,"Vajrahalli, Vaddarapalya - 560062, Karnataka",12.826000,77.543689,776114.30025,1.419255e+06,17720.045147,0,0
2,"Byatarayanadoddi, Anekal",12.825911,77.552895,777114.30025,1.419255e+06,17464.249197,0,0
3,"Byatarayanadoddi, Anekal",12.825821,77.562101,778114.30025,1.419255e+06,17262.676502,0,0
4,"Bannerughatta, Anekal",12.825731,77.571307,779114.30025,1.419255e+06,17117.242769,0,0


In [40]:
#blr_locations['Amenities Close By'] = np.sum(location_profile,axis=1)
ofc_cnt = []
hm_cnt =[]
shp_cnt = []
col_cnt = []

for i in range (0,len(location_profile)):
    ofc_cnt.append(location_profile[i][0])
    hm_cnt.append( location_profile[i][1])
    shp_cnt.append(location_profile[i][2])
    col_cnt.append(location_profile[i][3])

In [41]:
blr_locations['Offices'] = ofc_cnt
blr_locations['Residential'] = hm_cnt
blr_locations['Commercial']  = shp_cnt
blr_locations['Colleges']   = col_cnt
blr_locations.head()

,Address,Latitude,Longitudes,X,Y,Distance from center,Gyms in the Area,Yoga Centers in Area,Offices,Residential,Commercial,Colleges
0,"Bannerughatta, Anekal",12.816517,77.589626,781114.30025,1.418255e+06,18000.000000,0,0,0,0,0,0
1,"Vajrahalli, Vaddarapalya - 560062, Karnataka",12.826000,77.543689,776114.30025,1.419255e+06,17720.045147,0,0,0,0,0,0
2,"Byatarayanadoddi, Anekal",12.825911,77.552895,777114.30025,1.419255e+06,17464.249197,0,0,0,0,0,0
3,"Byatarayanadoddi, Anekal",12.825821,77.562101,778114.30025,1.419255e+06,17262.676502,0,0,0,0,0,0
4,"Bannerughatta, Anekal",12.825731,77.571307,779114.30025,1.419255e+06,17117.242769,0,0,0,0,0,0


In [42]:
create_download_link(blr_locations,"Download Link","blr_locations.csv")

In [43]:
categories_ = [OFFICE,HOME,COMMERCE,COLLEGE]
map_bangalore = folium.Map(location=blr_center, zoom_start=13)
folium.Marker(blr_center, popup='Bangalore').add_to(map_bangalore)
for gym in gyms.values():
    lat = gym[3]
    lon = gym[4]
    folium.CircleMarker([lat, lon], radius=3, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_bangalore)
for yog in yoga_centers.values():
    lat = yog[3]
    lon = yog[4]
    folium.CircleMarker([lat, lon], radius=3, color='red', fill=True, fill_color='red', fill_opacity=1).add_to(map_bangalore)

for cat in categories_:
    for lat,lon in zip(blr_profile[cat]['lat'],blr_profile[cat]['lng']):
          folium.CircleMarker([lat, lon], radius=3, color='green', fill=True, fill_color='red', fill_opacity=1).add_to(map_bangalore)
#folium.GeoJson(blr_local,style_function=bbmp,name='geojson').add_to(map_bangalore)

map_bangalore

In [50]:
map_bangalore = folium.Map(location=blr_center, zoom_start=13)
folium.Marker(blr_center, popup='Bangalore').add_to(map_bangalore)
for i in blr_locations.index:
    clr = 'red'
    if blr_locations['Gyms in the Area'][i] == 0 and blr_locations['Yoga Centers in Area'][i] == 0:
        clr = 'yellow'
        if  blr_locations['Offices'][i] > 0 or blr_locations['Residential'][i] > 0 or blr_locations['Colleges'][i] or blr_locations['Commercial'][i]:
                clr = 'green'
    folium.Circle([blr_locations['Latitude'][i], blr_locations['Longitudes'][i]],radius=500,color=clr, fill=True, fill_opacity=0.3).add_to(map_bangalore)
map_bangalore   

So now we have a view of all Gyms and Yoga Centers in the City. Also we have an overall profile of locations which have the 
amenities of our choice.

The above folium map denotes areas in red which already have a gym or a yoga center. 
The areas marked in green dont have either but have at least one of the amenities of our choice, 
thus would be investigated further
The areas marked in yellow are ones without any fitness center but dont have any amenities either. So could be ignored.
But we still investigate firther as some some pockets of these zones may still be in close vicinity of an amentity. 

This concludes the data gathering phase and we shall use the above to 